# Statistical foundations of Machine Learning

## INFO-F-422 TP: Ensembles of models and feature selection

Yann-Aël Le Borgne, Fabrizio Carcillo and Gianluca Bontempi

May 2, 2017

## Overview

Ensembles of models and feature selection are two machine learning techniques which can be used to improve the accuracy of preditions. 

Ensembles of models consist in building several predictive models using resampled subsets of the original training set. The method works particularly well for predictive models with high variance (for example, decision trees or neural networks). The average prediction of the resulting models usually strongly decreases the variance component of the error, and as a consequence improves the prediction accuracy. 

Feature selection aims at reducing the dimensionality of the problem, and is useful when input variables contain redundant or irrelevant (noisy) information. Benefits are twofold: it decreases the training time by simplifying the problem, and it decreases the complexity of the predictive model. This in turn usually improves the prediction accuracy, since high-dimensionality makes predictive models more prone to overfitting, and estimates of parameters more variant. 

In this session, we will illustrate both techniques using the IMDB 5000 dataset, which contains 27 variables describing 5043 movies. The variables contain information about the director, actors, number of Facebook likes for each actor, duration, genre, language, country, etc... We will use them to predict the movie success (through the IMDB score). The dataset together with a description of the variables is at https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset.

The dataset is on the github of the course, in datasets/movie_metadata.csv

## Data overview and preprocessing

Let us load and select a random subset of 1000 movies

In [4]:
data<-read.csv("datasets/movie_metadata.csv")
set.seed(2)
data<-data[sample(nrow(data),1000),]

Warning message in file(file, "rt"):
"impossible d'ouvrir le fichier 'datasets/movie_metadata.csv' : No such file or directory"

ERROR: Error in file(file, "rt"): impossible d'ouvrir la connexion


### Dataset overview

In [ ]:
dim(data)


In [ ]:
options(repr.matrix.max.cols=50)
data[1:2,]

In [ ]:
summary(data)

We see there is a mix of categorical and numerical variables, and some missing values. In order to simplify the analysis, let us remove the categorical variables, and replace the NA values with the mean values of the variables.

### Remove categorical variables

Check the type of input variables

In [ ]:
sapply(data[1,],class)

Get indices of categorical (factor) variables

In [ ]:
factor_variables<-which(sapply(data[1,],class)=="factor")
factor_variables

Remove categorical variables

In [ ]:
data_preprocessed<-data[,-factor_variables]
summary(data_preprocessed)

### Replace NA values with mean values

In [ ]:
replace_na_with_mean_value<-function(vec) {
    mean_vec<-mean(vec,na.rm=T)
    vec[is.na(vec)]<-mean_vec
    vec
}

In [ ]:
data_preprocessed<-data.frame(apply(data_preprocessed,2,replace_na_with_mean_value))
summary(data_preprocessed)

### Input and output variables

The output variable (Y) is the `imdb_score`, and all other variables (X) are considered as inputs.

In [ ]:
set.seed(3)

X<-data_preprocessed[,setdiff(colnames(data_preprocessed),"imdb_score")]
Y<-data_preprocessed[,"imdb_score"]

N<-nrow(X)    #Number of examples
n<-ncol(X)    #Number of input variables


Distribution of the `imdb_score`

In [ ]:
hist(Y)

In [ ]:
mean(Y)

In [ ]:
var(Y)

### 1) Modelling with linear and decision tree models

#### Linear model

* Let us create a linear model for predicting the IMDB score on the basis of the other variables, and compute its empricial mean square error

In [ ]:
DS<-cbind(X,imdb_score=Y)
    
model<- lm(imdb_score~.,DS) ### IMDB score given all the other ones (~.) over the dataset DS

Y.hat<- predict(model,X)
        
empirical_error<-mean((Y.hat-Y)^2) ### MSE for prediction of that model.

print(paste("Empirical error=",round(empirical_error,digits=4)))



* Which input variables are statistically correlated with the output?

In [ ]:
summary(model)

* Compute the validation error with a 10-fold cross-validation

In [ ]:
size.CV<-floor(N/10)

CV.err<-numeric(10)

for (i in 1:10) {
     i.ts<-(((i-1)*size.CV+1):(i*size.CV))  ### i.ts = indices of the test set for the i-th fold
     X.ts<-X[i.ts,]  
     Y.ts<-Y[i.ts]  
     
     i.tr<-setdiff(1:N,i.ts)                ###i.tr = indices of the training set for the i-th fold
     X.tr<-X[i.tr,]
     Y.tr<-Y[i.tr]                          
     
     DS<-cbind(X.tr,imdb_score=Y.tr)
    
     model<- lm(imdb_score~.,DS)      # create model with the training set
        
     Y.hat.ts<- predict(model,X.ts)  # predict value for the test set
        
     CV.err[i]<-mean((Y.hat.ts-Y.ts)^2)  # MSE for test set
}
    

print(paste("CV error=",round(mean(CV.err),digits=4), " ; std dev=",round(sd(CV.err),digits=4)))



#### Decision tree

* Modify the previous code to compute the empirical error using a decision tree model. Use the rpart package (see `?rpart` for help)

In [ ]:
library(rpart)       ### Run install.packages("rpart") to install
?rpart

In [ ]:
DS<-cbind(X,imdb_score=Y)

model<- rpart(imdb_score~.,DS)
        
Y.hat<- predict(model,X)
        
empirical_error<-mean((Y.hat-Y)^2) 

print(paste("Empirical error=",round(empirical_error,digits=4)))



* Plot the resulting tree using the `prp` function from the library `rpart.plot`

In [ ]:
library(rpart.plot)  ### Run install.packages("rpart.plot") to install

In [ ]:
prp(model)

* What is the 10-fold cross-validation error using a decision tree model?

In [ ]:
size.CV<-floor(N/10)

CV.err<-numeric(10)

for (i in 1:10) {
     i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
     X.ts<-X[i.ts,]  
     Y.ts<-Y[i.ts]  
     
     i.tr<-setdiff(1:N,i.ts)                
     X.tr<-X[i.tr,]
     Y.tr<-Y[i.tr]                          
     
     DS<-cbind(X.tr,imdb_score=Y.tr)
    
     model<- rpart(imdb_score~.,DS)
        
     Y.hat.ts<- predict(model,X.ts)
        
     CV.err[i]<-mean((Y.hat.ts-Y.ts)^2)
    }
    

print(paste("CV error=",round(mean(CV.err),digits=4), " ; std dev=",round(sd(CV.err),digits=4)))



## 2) Ensemble of models

Let us now create an ensemble of R linear models to make predictions. Complete the code below so that

* The training set is resampled before building a model
* The predictions of all model are averaged before testing

In [ ]:
size.CV<-floor(N/10)
R<-20   # R models

CV.err<-numeric(10)

for (i in 1:10) {
     i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
     X.ts<-X[i.ts,]  
     Y.ts<-Y[i.ts]  
     
     
     i.tr<-setdiff(1:N,i.ts)                
    
     Y.hat.ts.R<-matrix(0,nrow=nrow(X.ts),ncol=R)
    
     for (r in 1:R) {
         i.tr.resample<-sample(i.tr,rep=T)  #rep = replace
         X.tr<-X[i.tr.resample,]
         Y.tr<-Y[i.tr.resample]                          
     
         DS<-cbind(X.tr,imdb_score=Y.tr)
    
         model<- lm(imdb_score~.,DS)
        
         Y.hat.ts.R[,r]<- predict(model,X.ts)
     
     }
    
     Y.hat.ts<-apply(Y.hat.ts.R,1,mean)  #function will be applied over rows (1)
     CV.err[i]<-mean((Y.hat.ts-Y.ts)^2)
     }

print(paste("CV error=",round(mean(CV.err),digits=4), " ; std dev=",round(sd(CV.err),digits=4)))


* Is the CV error lower than with a single linear model?
* Use a decision tree as the base model. Is the CV error lower?

## 3) Feature selection

Two are the main approaches to feature selection:


* **Filter methods:** they are preprocessing methods. They attempt to
assess the merits of features from the data, ignoring the effects of
the selected feature subset on the performance of the learning
algorithm. Examples are methods that select variables by ranking them
through compression techniques (like PCA), or by computing correlation or a more advanced similarity measure such as minimum redundancy maximum relevance (mRMR) with the output.

*  **Wrapper methods:** these methods assess subsets of variables
according to their usefulness to a given predictor. The method
conducts a search for a good subset using the learning algorithm
itself as part of the evaluation function. The problem boils 
down to a problem of stochastic state space search. Example
are the stepwise methods proposed in linear regression analysis.



### Filter methods

#### Correlation with the output

* The following code performs features selection by keeping the most correlated variables with the output. Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?


In [5]:
size.CV<-floor(N/10)

CV.err<-matrix(0,nrow=n,ncol=10)

for (i in 1:10) {
    i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
    X.ts<-X[i.ts,]  
    Y.ts<-Y[i.ts]  
     
    i.tr<-setdiff(1:N,i.ts)
    X.tr<-X[i.tr,]
    Y.tr<-Y[i.tr]
     
    correlation<-abs(cor(X.tr,Y.tr))
    ranking<-sort(correlation,dec=T,index.return=T)$ix
     
    for (nb_features in 1:n) {
        DS<-cbind(X.tr[,ranking[1:nb_features],drop=F],imdb_score=Y.tr)
        model<- lm(imdb_score~.,DS)
        
        Y.hat.ts<- predict(model,X.ts[,ranking[1:nb_features],drop=F])
        
        CV.err[nb_features,i]<-mean((Y.hat.ts-Y.ts)^2)
    }
}  

print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV.err,1,mean),digits=4), " ; std dev=",round(apply(CV.err,1,sd),digits=4)))



ERROR: Error in eval(expr, envir, enclos): objet 'N' introuvable


In [24]:
ranking

[1]  7  1 10 15  6  2 12  3 13 11  4  8  5  9 14

#### mRMR

* The following code performs features selection by using the mRMR approach (see slides 49-52 of the course http://uv.ulb.ac.be/pluginfile.php/874401/mod_resource/content/2/fsel.pdf). Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?


In [6]:
size.CV<-floor(N/10)

CV.err<-matrix(0,nrow=n,ncol=10)

for (i in 1:10) {
    i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
    X.ts<-X[i.ts,]  
    Y.ts<-Y[i.ts]  
     
    i.tr<-setdiff(1:N,i.ts)
    X.tr<-X[i.tr,]
    Y.tr<-Y[i.tr]
    
    
    correlation<-abs(cor(X.tr,Y.tr))
    
    selected<-c()
    candidates<-1:n
    
    #mRMR ranks the variables by taking account not only the correlation with the output, but also by avoiding redudant variables
    for (j in 1:n) {
        redudancy.score<-numeric(length(candidates))
        if (length(selected)>0) {
            cor.selected.candidates<-cor(X.tr[,selected,drop=F],X.tr[,candidates,drop=F])
            redudancy.score<-apply(cor.selected.candidates,2,mean)
        }
        
        mRMR.score<-correlation[candidates]-redudancy.score
        
        selected_current<-candidates[which.max(mRMR.score)]
        selected<-c(selected,selected_current)
        candidates<-setdiff(candidates,selected_current)
    }
    
    ranking<-selected
     
    for (nb_features in 1:n) {
        DS<-cbind(X.tr[,ranking[1:nb_features],drop=F],imdb_score=Y.tr)
        model<- lm(imdb_score~.,DS)
        
        Y.hat.ts<- predict(model,X.ts[,ranking[1:nb_features],drop=F])
        
        CV.err[nb_features,i]<-mean((Y.hat.ts-Y.ts)^2)
    }
}  

print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV.err,1,mean),digits=4), " ; std dev=",round(apply(CV.err,1,sd),digits=4)))



ERROR: Error in eval(expr, envir, enclos): objet 'N' introuvable


In [26]:
selected

[1]  7 12  2  9  3  1 14 10 15  5  6 13  4 11  8

#### PCA

* The following code performs features selection by first transforming the inputs using PCA, and then keeping the most relevant principal components in the model. Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?



In [27]:
size.CV<-floor(N/10)

CV.err<-matrix(0,nrow=n,ncol=10)

X_pca<-data.frame(prcomp(X,retx=T)$x)

for (i in 1:10) {
    i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
    X.ts<-X_pca[i.ts,]  
    Y.ts<-Y[i.ts]  
     
    i.tr<-setdiff(1:N,i.ts)
    X.tr<-X_pca[i.tr,]
    Y.tr<-Y[i.tr]
     
    for (nb_features in 1:n) {
        DS<-cbind(X.tr[,1:nb_features,drop=F],imdb_score=Y.tr)
        model<- lm(imdb_score~.,DS)
        
        Y.hat.ts<- predict(model,X.ts[,1:nb_features,drop=F])
        
        CV.err[nb_features,i]<-mean((Y.hat.ts-Y.ts)^2)
    }
}  

print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV.err,1,mean),digits=4), " ; std dev=",round(apply(CV.err,1,sd),digits=4)))


 [1] "#Features:  1  ; CV error= 1.2051  ; std dev= 0.1211" 
 [2] "#Features:  2  ; CV error= 1.199  ; std dev= 0.1282"  
 [3] "#Features:  3  ; CV error= 0.9963  ; std dev= 0.081"  
 [4] "#Features:  4  ; CV error= 1.0043  ; std dev= 0.0777" 
 [5] "#Features:  5  ; CV error= 1.0046  ; std dev= 0.0815" 
 [6] "#Features:  6  ; CV error= 1.0084  ; std dev= 0.0818" 
 [7] "#Features:  7  ; CV error= 1.0063  ; std dev= 0.0753" 
 [8] "#Features:  8  ; CV error= 0.9993  ; std dev= 0.0871" 
 [9] "#Features:  9  ; CV error= 1.002  ; std dev= 0.0925"  
[10] "#Features:  10  ; CV error= 1.0025  ; std dev= 0.095" 
[11] "#Features:  11  ; CV error= 0.9773  ; std dev= 0.1005"
[12] "#Features:  12  ; CV error= 0.9661  ; std dev= 0.1263"
[13] "#Features:  13  ; CV error= 0.9142  ; std dev= 0.1412"
[14] "#Features:  14  ; CV error= 0.9157  ; std dev= 0.1426"
[15] "#Features:  15  ; CV error= 0.9138  ; std dev= 0.1508"


### Wrapper method: Forward selection

* The following code performs features selection by using a forward selection method (See slide 20 in http://uv.ulb.ac.be/pluginfile.php/874401/mod_resource/content/1/fsel.pdf). Compare the results for linear models and decision trees. What are the smallest CV errors, and how many features were needed?


In [28]:
size.CV<-floor(N/10)

selected<-NULL

for (round in 1:n) { 
    candidates<-setdiff(1:n,selected)
    
    CV.err<-matrix(0,nrow=length(candidates),ncol=10)
    
    for (j in 1:length(candidates)) {
        features_to_include<-c(selected,candidates[j])
        
        for (i in 1:10) {
            i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
            X.ts<-X[i.ts,features_to_include,drop=F]  
            Y.ts<-Y[i.ts]  
     
            i.tr<-setdiff(1:N,i.ts)
            X.tr<-X[i.tr,features_to_include,drop=F]
            Y.tr<-Y[i.tr]
     
            DS<-cbind(X.tr,imdb_score=Y.tr)
            model<- lm(imdb_score~.,DS)
        
            Y.hat.ts<- predict(model,X.ts)
        
            CV.err[j,i]<-mean((Y.hat.ts-Y.ts)^2)
        }
    }
    CV.err.mean<-apply(CV.err,1,mean)
    CV.err.sd<-apply(CV.err,1,sd)
    selected_current<-which.min(CV.err.mean)              
    selected<-c(selected,candidates[selected_current])
    print(paste("Round ",round," ; Selected feature: ",candidates[selected_current]," ; CV error=",round(CV.err.mean[selected_current],digits=4), " ; std dev=",round(CV.err.sd[selected_current],digits=4)))

}
                   


[1] "Round  1  ; Selected feature:  7  ; CV error= 1.0018  ; std dev= 0.0678"
[1] "Round  2  ; Selected feature:  12  ; CV error= 0.9618  ; std dev= 0.081"
[1] "Round  3  ; Selected feature:  1  ; CV error= 0.9261  ; std dev= 0.0961"
[1] "Round  4  ; Selected feature:  10  ; CV error= 0.912  ; std dev= 0.1088"
[1] "Round  5  ; Selected feature:  11  ; CV error= 0.9062  ; std dev= 0.1085"
[1] "Round  6  ; Selected feature:  2  ; CV error= 0.9016  ; std dev= 0.1243"
[1] "Round  7  ; Selected feature:  14  ; CV error= 0.9004  ; std dev= 0.1337"
[1] "Round  8  ; Selected feature:  4  ; CV error= 0.9  ; std dev= 0.1361"
[1] "Round  9  ; Selected feature:  13  ; CV error= 0.8989  ; std dev= 0.1372"
[1] "Round  10  ; Selected feature:  6  ; CV error= 0.899  ; std dev= 0.1342"
[1] "Round  11  ; Selected feature:  3  ; CV error= 0.8992  ; std dev= 0.1333"
[1] "Round  12  ; Selected feature:  15  ; CV error= 0.8996  ; std dev= 0.1336"
[1] "Round  13  ; Selected feature:  9  ; CV error= 0.9017  ;

In [29]:
colnames(X)[selected]

[1] "num_voted_users"           "title_year"               
 [3] "num_critic_for_reviews"    "num_user_for_reviews"     
 [5] "budget"                    "duration"                 
 [7] "aspect_ratio"              "actor_3_facebook_likes"   
 [9] "actor_2_facebook_likes"    "gross"                    
[11] "director_facebook_likes"   "movie_facebook_likes"     
[13] "facenumber_in_poster"      "cast_total_facebook_likes"
[15] "actor_1_facebook_likes"

## Further preprocessing to add categorical variables

Categorical variables usually need to be transformed with 'one-hot-encoding' in order to be processed by a learning algorithm. That is, for each value of the categorical variable, a binary feature is created, which is set to one whenever that value is present. This can be done using the `dummy.data.frame` of the `dummies` package.

```
install.packages('dummies')
library(dummies)
```

In the following, we add some categorical variables to the peprocessing dataset. The set of categorical variables is

In [30]:
library(dummies)

dummies-1.5.6 provided by Decision Patterns



In [31]:
factor_variables

color   director_name    actor_2_name          genres    actor_1_name 
              1               2               7              10              11 
    movie_title    actor_3_name   plot_keywords movie_imdb_link        language 
             12              15              17              18              20 
        country  content_rating 
             21              22

Let us have an overview of the their content

In [32]:
data_factor<-data[,factor_variables]

In [33]:
dim(data_factor)

[1] 1000   12

In [34]:
data_factor[1:2,]

,color,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,movie_imdb_link,language,country,content_rating
933,Color,James L. Brooks,Yeardley Smith,Comedy|Drama|Romance,Lupe Ontiveros,As Good as It Gets,Shirley Knight,dog|friendship|neighbor|unlikely friendship|writer,http://www.imdb.com/title/tt0119822/?ref_=fn_tt_tt_1,English,USA,PG-13
3542,Color,Robert C. Cooper,Christopher Judge,Action|Adventure|Drama|Fantasy|Sci-Fi,Ben Browder,Stargate: The Ark of Truth,Julian Sands,2000s|evil god|space opera|stargate|wormhole,http://www.imdb.com/title/tt0942903/?ref_=fn_tt_tt_1,English,USA,


Let us keep four of them: Color, language, country and content_rating, and transform them with one-hot-encoding

In [35]:
variable_to_keep<-c("color","language","country","content_rating")

In [36]:
data_factor_onehot <- dummy.data.frame(data_factor[,variable_to_keep], sep="_")

In [37]:
dim(data_factor_onehot)

[1] 1000   76

In [38]:
data_factor_onehot[1:2,]

,color_,color_ Black and White,color_Color,language_,language_Aboriginal,language_Arabic,language_Bosnian,language_Cantonese,language_Dutch,language_English,language_French,language_German,language_Hebrew,language_Hindi,language_Icelandic,language_Italian,language_Japanese,language_Mandarin,language_Mongolian,language_Persian,language_Polish,language_Portuguese,language_Spanish,language_Swahili,language_Swedish,⋯,country_Peru,country_Poland,country_Russia,country_South Korea,country_Spain,country_Sweden,country_UK,country_USA,country_United Arab Emirates,content_rating_,content_rating_Approved,content_rating_G,content_rating_M,content_rating_NC-17,content_rating_Not Rated,content_rating_PG,content_rating_PG-13,content_rating_Passed,content_rating_R,content_rating_TV-14,content_rating_TV-G,content_rating_TV-MA,content_rating_TV-PG,content_rating_Unrated,content_rating_X
933,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3542,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


These could be added to the previously preprocessed dataset, and used to further improve the prediction accuracy using the feature selection/ensemble techniques seen above. 

In [39]:
data_preprocessed_extended<-cbind(data_preprocessed,data_factor_onehot)

In [40]:
dim(data_preprocessed_extended)

[1] 1000   92

In [41]:
summary(data_preprocessed_extended)

 num_critic_for_reviews    duration     director_facebook_likes
 Min.   :  1.0          Min.   : 11.0   Min.   :    0.0        
 1st Qu.: 50.0          1st Qu.: 93.0   1st Qu.:    7.0        
 Median :110.0          Median :104.0   Median :   47.5        
 Mean   :143.5          Mean   :107.7   Mean   :  735.7        
 3rd Qu.:199.0          3rd Qu.:118.0   3rd Qu.:  210.5        
 Max.   :813.0          Max.   :511.0   Max.   :22000.0        
 actor_3_facebook_likes actor_1_facebook_likes     gross          
 Min.   :    0.0        Min.   :     0.0       Min.   :     1332  
 1st Qu.:  123.8        1st Qu.:   591.8       1st Qu.:  9709388  
 Median :  366.0        Median :   984.5       Median : 39670256  
 Mean   :  618.1        Mean   :  6594.5       Mean   : 52916026  
 3rd Qu.:  635.5        3rd Qu.: 11000.0       3rd Qu.: 54422773  
 Max.   :23000.0        Max.   :260000.0       Max.   :533316061  
 num_voted_users   cast_total_facebook_likes facenumber_in_poster
 Min.   :     13 

## Using other predictive models

Other models could be used, for example support vector machines, neural networks, K-nearest neighbors (using the `svm`, `nnt`or `lazy` (! predict()$h ) functions from the `e1071`, `nnet` or `lazy` packages, respectively). Note that scaling the data is usually necessary when using neural networks and K-nearest neighbors approaches. 

In [ ]:
nnet (size=5)
  non linear function gives the output, and goes from 0 to 1. We want a grade between 0 and 10 so we need to give a linear fct at the output by using linout=T
  range of the values are quite different => nnet and nearrst neighbour are really bad when this happens so we have to normalize/scale the data scale(X)
scale gives a matrix and not a dataframe => X<-dataframe(scale(X)) (see function scale for more details on what it does)

see sample
    
    when you scale you have to use the mean and variance obtained from the scale of the training data for your test set.